In [1]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("1.4.1.pdf")
docs = loader.load()
docs

[Document(metadata={'source': '1.4.1.pdf', 'page': 0, 'page_label': '1'}, page_content='1.4.1Institutionobtainsfeedbackontheacademicperformanceandambienceoftheinstitutionfromvariousstakeholders,suchasStudents,Teachers,Employers,Alumnietc.andactiontakenreporton thefeedbackis madeavailableon Feedbackprocessesof theinstitutionmaybeclassifiedasfollows:A. Feedbackcollected,analysed,actiontaken& communicatedto therelevantbodiesandfeedbackhostedontheinstitutionalwebsiteB.Feedbackcollected,analysedandactionhasbeentakenandcommunicatedtotherelevantbodiesC.FeedbackcollectedandanalysedD.FeedbackcollectedE.Feedbacknotcollectedinstitutionalwebsite\nResponse:OptionA\n'),
 Document(metadata={'source': '1.4.1.pdf', 'page': 1, 'page_label': '2'}, page_content='1.4.1 Institution obtains feedback on the academic performance and ambience of the institution from various stakeholders, \nsuch as Students, Teachers, Employers, Alumni etc. and action taken report on the feedback is made available on Feedback \n

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
text_splitter.split_documents(docs)[:5]

[Document(metadata={'source': '1.4.1.pdf', 'page': 0, 'page_label': '1'}, page_content='1.4.1Institutionobtainsfeedbackontheacademicperformanceandambienceoftheinstitutionfromvariousstakeholders,suchasStudents,Teachers,Employers,Alumnietc.andactiontakenreporton thefeedbackis madeavailableon Feedbackprocessesof theinstitutionmaybeclassifiedasfollows:A. Feedbackcollected,analysed,actiontaken& communicatedto therelevantbodiesandfeedbackhostedontheinstitutionalwebsiteB.Feedbackcollected,analysedandactionhasbeentakenandcommunicatedtotherelevantbodiesC.FeedbackcollectedandanalysedD.FeedbackcollectedE.Feedbacknotcollectedinstitutionalwebsite\nResponse:OptionA'),
 Document(metadata={'source': '1.4.1.pdf', 'page': 1, 'page_label': '2'}, page_content='1.4.1 Institution obtains feedback on the academic performance and ambience of the institution from various stakeholders, \nsuch as Students, Teachers, Employers, Alumni etc. and action taken report on the feedback is made available on Feedback \npr

In [12]:
documents=text_splitter.split_documents(docs)
documents

[Document(metadata={'source': '1.4.1.pdf', 'page': 0, 'page_label': '1'}, page_content='1.4.1Institutionobtainsfeedbackontheacademicperformanceandambienceoftheinstitutionfromvariousstakeholders,suchasStudents,Teachers,Employers,Alumnietc.andactiontakenreporton thefeedbackis madeavailableon Feedbackprocessesof theinstitutionmaybeclassifiedasfollows:A. Feedbackcollected,analysed,actiontaken& communicatedto therelevantbodiesandfeedbackhostedontheinstitutionalwebsiteB.Feedbackcollected,analysedandactionhasbeentakenandcommunicatedtotherelevantbodiesC.FeedbackcollectedandanalysedD.FeedbackcollectedE.Feedbacknotcollectedinstitutionalwebsite\nResponse:OptionA'),
 Document(metadata={'source': '1.4.1.pdf', 'page': 1, 'page_label': '2'}, page_content='1.4.1 Institution obtains feedback on the academic performance and ambience of the institution from various stakeholders, \nsuch as Students, Teachers, Employers, Alumni etc. and action taken report on the feedback is made available on Feedback \npr

In [11]:
# from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

db=FAISS.from_documents(documents,OllamaEmbeddings())

In [14]:
db

In [15]:
query="Feedback processes of the institution may be classified as "
result=db.similarity_search(query)
result

[Document(id='50b282c4-44ab-4edb-946a-03607a2a433b', metadata={'source': '1.4.1.pdf', 'page': 0, 'page_label': '1'}, page_content='1.4.1Institutionobtainsfeedbackontheacademicperformanceandambienceoftheinstitutionfromvariousstakeholders,suchasStudents,Teachers,Employers,Alumnietc.andactiontakenreporton thefeedbackis madeavailableon Feedbackprocessesof theinstitutionmaybeclassifiedasfollows:A. Feedbackcollected,analysed,actiontaken& communicatedto therelevantbodiesandfeedbackhostedontheinstitutionalwebsiteB.Feedbackcollected,analysedandactionhasbeentakenandcommunicatedtotherelevantbodiesC.FeedbackcollectedandanalysedD.FeedbackcollectedE.Feedbacknotcollectedinstitutionalwebsite\nResponse:OptionA'),
 Document(id='aa829858-815b-4ac1-831a-11ea8fa76a56', metadata={'source': '1.4.1.pdf', 'page': 1, 'page_label': '2'}, page_content='8 Feedback on Curriculum by different stack Holders Link Yes\n9 Parents Feedback Link Yes'),
 Document(id='5d27ec37-bbce-4c39-8ef2-bab1b073d90a', metadata={'source

In [16]:
from langchain_community.llms import Ollama
## Load Ollama LAMA2 LLM model
llm=Ollama(model="llama2")
llm

Ollama()

In [17]:
## Design ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
I will tip you $1000 if the user finds the answer helpful. 
<context>
{context}
</context>
Question: {input}""")

In [18]:
## Chain Introduction
## Create Stuff Docment Chain

from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(llm,prompt)

In [19]:
retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002026AC3D520>, search_kwargs={})

In [20]:
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [21]:
response=retrieval_chain.invoke({"input":"Feedback processes of the institution may be classified as"})

In [22]:
response['answer']

'Based on the provided context, the feedback processes of the institution can be classified into four categories:\n\n1. Feedback collected, analysed, action taken & communicated to the relevant bodies and feedback hosted on the institutional website (Option A)\n2. Feedback collected, analysed, and action has been taken and communicated to the relevant bodies (Option B)\n3. Feedback collected and analysed (Option C)\n4. Feedback not collected on the institutional website (Option E)\n\nThe options are based on the information provided in the context, where the institution obtains feedback from various stakeholders, including students, teachers, employers, alumni, parents, and program and course exit feedback. The action taken report on the feedback is made available on the institutional website.'